In [1]:
# Algorithm reference from https://slidetodoc.com/beam-search-seminar-by-vinay-surana-08005031-vivek/
# The goal states for this algorithm is to find solution to zero pairs of attacking queens
# Start with N randomly initial states
# All the successors of all k states are generated
# The algorithm terminates into two cases; 1. if required goal is reached, stop.
# 2. If no node left for further exploration, stop. else select the k best successors from the complete list and repeat.
# Time library is used to to measure LBS algorithm time in seconds
# K beam width is implemented.
# user input for the N board size start at index 0 (such if N = 8, hence 7)
from random import randint
import time

N_size = int(input("Enter N board size : "))
# N_Kbeam_Width = int(input("Enter K beam width: "))

start_time = time.time()


def random_value(N_size):
    return [randint(0, N_size) for _ in range(N_size + 1)]


randstate = random_value(N_size)


# -------------------------------------------------------------------------------------
# function to detect attack in row, column and diagonal
# Code adapted from https://github.com/davpal/eight-queens/blob/master/queens.py
def heuristic(individual):
    horizontal_collisions = sum([individual.count(queen) - 1 for queen in individual]) / 2

    n = len(individual)
    left_diagonal = [0] * 2 * n
    right_diagonal = [0] * 2 * n
    for i in range(n):
        left_diagonal[i + individual[i] - 1] += 1
        right_diagonal[len(individual) - i + individual[i] - 2] += 1

    diagonal_collisions = 0
    for i in range(2 * n - 1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i] - 1
        if right_diagonal[i] > 1:
            counter += right_diagonal[i] - 1
        diagonal_collisions += counter  # / (n-abs(i-n+1))
    return int(horizontal_collisions + diagonal_collisions)

    # count += 1


# return count

# ---------------------Beam search algorithm start here------------------------#
def nd_successor(state, successors):
    count = 0
    for i in range(len(state)):  # find len of state such initial state = 8 ; hence i is 01234567
        move = int(state[i])  # move the int from initial state one by one
        change_list = list(range(len(state)))  # [0, 1, 2, 3, 4, 5, 6, 7] from initial state of number entered
        change_list.pop(
            move)  # .remove; remove the one of the value in the list result output :None (doesn't return any value)
        change_tuple = tuple(change_list)
        # print("check",change_list.pop(move))

        for j in change_tuple:

            neighbour = tuple((state[:i])) + tuple([j]) + tuple((state[i + 1:]))
            # print("1tup",tuple((state[:i])))
            # print("2tup", tuple([j]))
            # print("3tup", tuple((state[i + 1:])))
            successors[neighbour] = heuristic(neighbour)
            count += 1
            # h = heuristic(neighbour)
            # break once find goal state
            if heuristic(neighbour) == 0:
                # print("heuristic_neighbour",heuristic(neighbour))
                break
        else:
            continue
        break
    return successors, count  # here the count is the total number of nodes


def local_beam_search(k_beam_width):
    Continue = True
    # This is the total number of nodes visited; assign value to 0
    n_node = 0
    # The total steps(iterations) takes before terminate local beam search; assigned to default value to 0
    n_step = 0
    # list to store goal state
    goal_state = []

    randomInitialState = randstate
    successors = {}
    for i in range(k_beam_width):  # K is the numbers of beam width and i is the number starting 0 to 9
        # generate randomize number based on i in range of K beam width
        initial = randomInitialState
        successors[tuple(initial)] = heuristic(initial)

    if 0 in successors.values():  # return the values found in successors
        # Continue = False
        for state, conflict in successors.items():  # return tuple containing two values from state and vio
            if conflict == 0:
                goal_state.append(state)
    previous_minimum_heuristic_value = min(successors.values())  # get only minimum heuristic value found

    no_goal_state_list = []  # List to store the value when the LBS algorithm can't find solution to the goal state
    while Continue:  # Continue = True
        current_successors = {}
        for successor in successors:
            current_successors, count = nd_successor(successor, current_successors)
            n_node += count
        # -------------------------------------------------------------------------------------
        # select the K best (heuristic value)
        # Reference https://www.programiz.com/python-programming/dictionary
        # https://www.w3schools.com/python/ref_func_sorted.asp(for implementing dictionary key)
        successors_k_best = sorted(current_successors, key=current_successors.get)[:k_beam_width]  # sorted according
        # to ascending order by default
        print("successor :", successors_k_best)
        # stored current successor in dictionary based on the current successor with minimum heuristic found
        successors = {key: current_successors[key] for key in successors_k_best}
        print("successor min heuristic value found: ", successors)
        current_minimum_heuristic_value = min(successors.values())  # get only minimum heuristic value from successors
        n_step += 1  # for step of iteration
        print("Step", n_step)
        # select the K best (heuristic value) end here
        # -------------------------------------------------------------------------------------
        # termination when finding goal state
        if 0 in successors.values():
            for state, conflict in successors.items():
                if conflict == 0:
                    goal_state.append(state)
        # -------------------------------------------------------------------------------------
        # termination when no further nodes exploration
        if previous_minimum_heuristic_value <= current_minimum_heuristic_value:
            Continue = False
            for state, conflict in successors.items():
                if conflict == current_minimum_heuristic_value:
                    no_goal_state_list.append(state)
        previous_minimum_heuristic_value = min(current_minimum_heuristic_value, previous_minimum_heuristic_value)
        print("minimum heuristic found:", previous_minimum_heuristic_value)
        # -------------------------------------------------------------------------------------
    printResult = True
    if printResult:
        print('Local Beam Search, K Beam Width = ', k_beam_width)
        # if no goal states is found, print below
        if len(goal_state) == 0:
            print('Initial board:', randstate)
            print(
                '{} nodes were visited. No goal state found in {} last iteration from the lowest heuristic value.'.format(
                    n_node, n_step))
            print('Local Beam search trapped, found {} pairs of attacking queens'.format(
                previous_minimum_heuristic_value), 'from',
                no_goal_state_list)  # here pair containing 2 queens
            print('Time taken (s): '"%s seconds" % (time.time() - start_time), '\n')
        # else print below
        else:
            print('Initial board:', randstate)
            print(
                '{} nodes were visited. {} goal state found in {} last iteration from the lowest heuristic value'.format(
                    n_node, len(goal_state), n_step))
            print('Goal state (Non-attacking queens):', goal_state)
            print('Time taken (s): '"%s seconds" % (time.time() - start_time), '\n')
    else:
        return len(goal_state), previous_minimum_heuristic_value
    # -------------------------------------------------------------------------------------
    # Input the k beam size and run the LBS algorithm start here


# print('Run Local Beam Search for k beam width = {}'.format(N_Kbeam_Width))
K1 = 2
K2 = 10
K3 = 15
k4 = 50
local_beam_search(K1)
local_beam_search(K2)
local_beam_search(K3)
local_beam_search(k4)

# user input for the N board size start at index 0 (such if N = 8, hence 7)

Enter N board size : 14
successor : [(7, 3, 11, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 9, 11, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10)]
successor min heuristic value found:  {(7, 3, 11, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10): 11, (7, 9, 11, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10): 11}
Step 1
minimum heuristic found: 11
successor : [(8, 3, 11, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 0, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10)]
successor min heuristic value found:  {(8, 3, 11, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10): 9, (7, 3, 0, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10): 9}
Step 2
minimum heuristic found: 9
successor : [(8, 3, 0, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (8, 3, 1, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10)]
successor min heuristic value found:  {(8, 3, 0, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10): 7, (8, 3, 1, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10): 7}
Step 3
minimum heuristic found: 7
successor : [(8, 3, 0, 4, 7, 0, 11, 11, 14,

successor : [(8, 1, 7, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (8, 1, 13, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (8, 1, 14, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (8, 1, 11, 4, 6, 0, 0, 5, 14, 2, 13, 3, 9, 12, 10), (8, 1, 11, 4, 6, 0, 10, 5, 14, 2, 13, 3, 9, 12, 10), (8, 1, 11, 4, 6, 0, 13, 5, 14, 2, 13, 3, 9, 12, 10), (8, 1, 11, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 1, 11, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (4, 1, 11, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (7, 1, 11, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10)]
successor min heuristic value found:  {(8, 1, 7, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10): 1, (8, 1, 13, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10): 1, (8, 1, 14, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10): 1, (8, 1, 11, 4, 6, 0, 0, 5, 14, 2, 13, 3, 9, 12, 10): 1, (8, 1, 11, 4, 6, 0, 10, 5, 14, 2, 13, 3, 9, 12, 10): 1, (8, 1, 11, 4, 6, 0, 13, 5, 14, 2, 13, 3, 9, 12, 10): 1, (8, 1, 11, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10): 1, (8, 1, 11, 4, 6, 0, 11, 5, 14, 2

successor : [(7, 1, 11, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (7, 1, 10, 4, 6, 0, 11, 11, 14, 2, 13, 3, 9, 12, 10), (7, 1, 10, 4, 6, 0, 11, 11, 14, 2, 13, 5, 9, 12, 10), (8, 1, 12, 4, 6, 0, 11, 11, 14, 2, 13, 2, 9, 12, 10), (7, 1, 12, 4, 6, 0, 11, 5, 14, 2, 13, 2, 9, 12, 10), (7, 1, 12, 4, 6, 0, 11, 11, 14, 3, 13, 2, 9, 12, 10), (7, 1, 12, 4, 6, 0, 11, 11, 14, 2, 13, 5, 9, 12, 10), (7, 1, 12, 4, 6, 0, 11, 11, 14, 2, 13, 2, 9, 3, 10), (8, 1, 13, 4, 6, 0, 11, 11, 14, 2, 13, 2, 9, 12, 10), (7, 1, 13, 4, 6, 0, 11, 5, 14, 2, 13, 2, 9, 12, 10), (7, 1, 13, 4, 6, 0, 11, 11, 14, 3, 13, 2, 9, 12, 10), (7, 1, 13, 4, 6, 0, 11, 11, 14, 2, 13, 3, 9, 12, 10), (7, 1, 13, 4, 6, 0, 11, 11, 14, 2, 13, 5, 9, 12, 10), (8, 1, 14, 4, 6, 0, 11, 11, 14, 2, 13, 2, 9, 12, 10), (7, 1, 14, 4, 6, 0, 11, 5, 14, 2, 13, 2, 9, 12, 10)]
successor min heuristic value found:  {(7, 1, 11, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10): 2, (7, 1, 10, 4, 6, 0, 11, 11, 14, 2, 13, 3, 9, 12, 10): 3, (7, 1, 10, 4, 6, 0, 11, 11, 14

successor : [(7, 12, 11, 4, 6, 0, 11, 11, 14, 2, 13, 2, 9, 12, 10), (8, 3, 11, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 0, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 1, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 6, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 8, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 10, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 13, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 14, 4, 7, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 11, 4, 1, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 11, 4, 5, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 11, 4, 6, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 11, 4, 8, 0, 11, 11, 14, 2, 13, 2, 12, 12, 10), (7, 3, 11, 4, 7, 0, 11, 1, 14, 2, 13, 2, 12, 12, 10), (7, 3, 11, 4, 7, 0, 11, 5, 14, 2, 13, 2, 12, 12, 10), (7, 3, 11, 4, 7, 0, 11, 11, 14, 1, 13, 2, 12, 12, 10), (7, 3, 11, 4, 7, 0, 11, 11, 14, 6, 13, 2, 12, 12, 10), (7, 3, 11, 4, 7, 0, 11, 11, 14, 2, 13, 1, 12, 12, 10), (7,

successor : [(7, 1, 10, 4, 6, 0, 11, 11, 14, 2, 13, 2, 9, 12, 10), (7, 1, 12, 4, 6, 0, 11, 11, 14, 2, 13, 2, 9, 12, 10), (7, 1, 13, 4, 6, 0, 11, 11, 14, 2, 13, 2, 9, 12, 10), (7, 1, 14, 4, 6, 0, 11, 11, 14, 2, 13, 2, 9, 12, 10), (7, 1, 11, 4, 6, 0, 11, 5, 14, 2, 13, 2, 9, 12, 10), (7, 1, 11, 4, 6, 0, 11, 11, 14, 2, 13, 3, 9, 12, 10), (7, 1, 11, 4, 6, 0, 11, 11, 14, 2, 13, 5, 9, 12, 10), (7, 12, 0, 4, 6, 0, 11, 11, 14, 2, 13, 1, 9, 12, 10), (7, 12, 0, 4, 6, 0, 11, 11, 14, 2, 13, 3, 9, 12, 10), (7, 12, 0, 4, 6, 0, 11, 11, 14, 2, 13, 5, 9, 12, 10), (7, 12, 1, 4, 6, 0, 11, 11, 14, 2, 13, 3, 9, 12, 10), (7, 12, 1, 4, 6, 0, 11, 11, 14, 2, 13, 5, 9, 12, 10), (7, 12, 1, 4, 6, 0, 11, 11, 14, 2, 13, 2, 9, 3, 10), (7, 12, 10, 4, 6, 0, 11, 11, 14, 2, 13, 3, 9, 12, 10), (7, 12, 10, 4, 6, 0, 11, 11, 14, 2, 13, 5, 9, 12, 10), (7, 12, 14, 4, 6, 0, 11, 11, 14, 2, 13, 1, 9, 12, 10), (7, 12, 14, 4, 6, 0, 11, 11, 14, 2, 13, 3, 9, 12, 10), (7, 5, 11, 4, 6, 0, 11, 1, 14, 2, 13, 2, 9, 12, 10), (7, 12, 11, 4,

successor : [(8, 1, 11, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (8, 12, 1, 4, 6, 0, 11, 11, 14, 2, 13, 3, 9, 7, 10), (4, 1, 11, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (7, 1, 6, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (7, 1, 13, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (7, 1, 14, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (7, 1, 11, 13, 6, 0, 11, 5, 14, 2, 13, 3, 9, 12, 10), (7, 1, 11, 4, 6, 0, 0, 5, 14, 2, 13, 3, 9, 12, 10), (7, 1, 11, 4, 6, 0, 10, 5, 14, 2, 13, 3, 9, 12, 10), (7, 1, 11, 4, 6, 0, 14, 5, 14, 2, 13, 3, 9, 12, 10), (0, 5, 11, 4, 6, 0, 11, 1, 14, 2, 13, 3, 9, 12, 10), (8, 5, 11, 4, 6, 0, 11, 1, 14, 2, 13, 3, 9, 12, 10), (12, 5, 11, 4, 6, 0, 11, 1, 14, 2, 13, 3, 9, 12, 10), (7, 5, 0, 4, 6, 0, 11, 1, 14, 2, 13, 3, 9, 12, 10), (7, 5, 2, 4, 6, 0, 11, 1, 14, 2, 13, 3, 9, 12, 10), (7, 5, 10, 4, 6, 0, 11, 1, 14, 2, 13, 3, 9, 12, 10), (7, 5, 13, 4, 6, 0, 11, 1, 14, 2, 13, 3, 9, 12, 10), (7, 5, 14, 4, 6, 0, 11, 1, 14, 2, 13, 3, 9, 12, 10), (7, 5, 11, 1, 6, 0, 11, 1, 14, 2, 1

successor : [(9, 5, 2, 4, 6, 0, 11, 1, 14, 7, 13, 8, 3, 12, 10), (0, 12, 1, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (4, 12, 1, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (9, 12, 1, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 1, 1, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 5, 1, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 8, 1, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 14, 1, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 2, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 14, 4, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 1, 3, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 1, 12, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 1, 13, 6, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 1, 4, 11, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 1, 4, 14, 0, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 1, 4, 6, 14, 11, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 1, 4, 6, 0, 10, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 1, 4, 6, 0, 13, 5, 14, 2, 13, 3, 9, 7, 10), (8, 12, 1, 4, 6, 0, 11, 11, 14, 2, 13, 3, 9, 7, 